In [2]:
# For an example date, match the wind data with each video according to the media data
# Print out the results for debugging
####################################################################

import os
import pandas as pd
import mysql.connector

def extract_time_minutes_seconds(timestr):
    mins, secs = timestr.split('m')
    return int(mins.strip()), int(secs.split('s')[0].strip())

media_folder = '\\sf3.bss.phy.private.cam.ac.uk\\cicutagroup\\crop_data\\airdata\\media_data'
wind_folder = '\\sf3.bss.phy.private.cam.ac.uk\\cicutagroup\\crop_data\\airdata\\wind_data'

# 1. Gather all related files for a specific date
date_str = "Jul_9th__2022"
media_files = [f for f in os.listdir(media_folder) if date_str in f]
wind_files = [f for f in os.listdir(wind_folder) if date_str in f]
# Sort the files by name
media_files.sort()
wind_files.sort()
    

all_wind_data = []

# 2. Extract wind data for each video
for media_file in media_files:
    df_media = pd.read_csv(os.path.join(media_folder, media_file))
    
    video_starts = df_media[df_media['Type'] == '    Video Started    ']['Flight time'].tolist()
    video_ends = df_media[df_media['Type'] == '    Video Ended    ']['Flight time'].tolist()

    df_wind = pd.read_csv(os.path.join(wind_folder, media_file))
    video_wind_data_list = []
    for start, end in zip(video_starts, video_ends):
        start_min, start_sec = extract_time_minutes_seconds(start)
        end_min, end_sec = extract_time_minutes_seconds(end)

        df_wind['TimeInMinutes'] = df_wind['Flight time'].apply(lambda x: extract_time_minutes_seconds(x)[0])
        df_wind['TimeInSeconds'] = df_wind['Flight time'].apply(lambda x: extract_time_minutes_seconds(x)[1])

        filtered_wind_data = df_wind[
            (df_wind['TimeInMinutes'] * 60 + df_wind['TimeInSeconds'] >= start_min * 60 + start_sec - 1) &
            (df_wind['TimeInMinutes'] * 60 + df_wind['TimeInSeconds'] <= end_min * 60 + end_sec + 1)
        ]
        video_wind_data_list.append(filtered_wind_data)

    all_wind_data.extend(video_wind_data_list)

# 3. Fetch all videos from the database
cnx = mysql.connector.connect(user='root', password='dronevideos', host='localhost', database='crop_videos')
cursor = cnx.cursor()

import re

def format_date_str(date_str):
    """Convert date string in format "Mon_DDth__YYYY" to "YYYY-MM-DD"."""
    match = re.match(r"(\w{3})_(\d{1,2})(?:st|nd|rd|th)__(\d{4})", date_str)
    month_str_to_num = {
        'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04', 'May': '05', 'Jun': '06',
        'Jul': '07', 'Aug': '08', 'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'
    }

    if match:
        month = month_str_to_num[match.group(1)]
        day = match.group(2).zfill(2)  # Making sure it's two digits
        year = match.group(3)
        return f"{year}-{month}-{day}"
    else:
        raise ValueError("Invalid date string format")

# Example Usage
formatted_date = format_date_str(date_str)
query = f"SELECT * FROM video WHERE create_date='{formatted_date}' ORDER BY create_time ASC"
cursor.execute(query)

videos_from_db = cursor.fetchall()

# 4. Map wind data with videos
mapped_data = []
for video, wind_data in zip(videos_from_db, all_wind_data):
    mapped_data.append((video, wind_data))

# Print the result
for video, wind_data in mapped_data:
    video_info = f"Video created on {video[1]} at {video[2]} with duration {video[3]}"
    print(video_info)
    print(wind_data[['Flight time', 'Altitude', 'Wind Direction', 'Wind Speed']])
    print("-" * 40)


Video created on DJI_0004.MP4 at \\sf3.bss.phy.private.cam.ac.uk\cicutagroup\crop_data\wheat_videos\drone_videos\original\DJI_0004.MP4 with duration 2022-07-09
   Flight time Altitude  Wind Direction Wind Speed
14     02m 20s    6.8 m             335   3.02 m/s
15     02m 25s    6.8 m             347   2.99 m/s
16     02m 30s    6.6 m             337   3.27 m/s
----------------------------------------
Video created on DJI_0012.MP4 at \\sf3.bss.phy.private.cam.ac.uk\cicutagroup\crop_data\wheat_videos\drone_videos\original\DJI_0012.MP4 with duration 2022-07-09
   Flight time Altitude  Wind Direction Wind Speed
18     02m 45s    3.6 m             332   2.73 m/s
19     02m 50s    3.7 m             340   2.37 m/s
20     02m 55s    3.8 m             326   1.77 m/s
----------------------------------------
Video created on DJI_0013.MP4 at \\sf3.bss.phy.private.cam.ac.uk\cicutagroup\crop_data\wheat_videos\drone_videos\original\DJI_0013.MP4 with duration 2022-07-09
   Flight time Altitude  Wind 

In [4]:
# For an example date, match the wind data with each video according to the media data
# Update the wind data into the database and link them to the videos
####################################################################

import os
import pandas as pd
import mysql.connector
import numpy as np

def extract_time_minutes_seconds(timestr):
    mins, secs = timestr.split('m')
    return int(mins.strip()), int(secs.split('s')[0].strip())

media_folder = '\\sf3.bss.phy.private.cam.ac.uk\\cicutagroup\\crop_data\\airdata\\media_data'
wind_folder = '\\sf3.bss.phy.private.cam.ac.uk\\cicutagroup\\crop_data\\airdata\\wind_data'

# 1. Gather all related files for a specific date
date_str = "Jul_9th__2022"
media_files = [f for f in os.listdir(media_folder) if date_str in f]
wind_files = [f for f in os.listdir(wind_folder) if date_str in f]
# Sort the files by name
media_files.sort()
wind_files.sort()
    

all_wind_data = []

# 2. Extract wind data for each video
for media_file in media_files:
    df_media = pd.read_csv(os.path.join(media_folder, media_file))
    
    video_starts = df_media[df_media['Type'] == '    Video Started    ']['Flight time'].tolist()
    video_ends = df_media[df_media['Type'] == '    Video Ended    ']['Flight time'].tolist()

    df_wind = pd.read_csv(os.path.join(wind_folder, media_file))
    video_wind_data_list = []
    for start, end in zip(video_starts, video_ends):
        start_min, start_sec = extract_time_minutes_seconds(start)
        end_min, end_sec = extract_time_minutes_seconds(end)

        df_wind['TimeInMinutes'] = df_wind['Flight time'].apply(lambda x: extract_time_minutes_seconds(x)[0])
        df_wind['TimeInSeconds'] = df_wind['Flight time'].apply(lambda x: extract_time_minutes_seconds(x)[1])

        filtered_wind_data = df_wind[
            (df_wind['TimeInMinutes'] * 60 + df_wind['TimeInSeconds'] >= start_min * 60 + start_sec - 1) &
            (df_wind['TimeInMinutes'] * 60 + df_wind['TimeInSeconds'] <= end_min * 60 + end_sec + 1)
        ]
        video_wind_data_list.append(filtered_wind_data)

    all_wind_data.extend(video_wind_data_list)

# 3. Fetch all videos from the database
cnx = mysql.connector.connect(user='root', password='dronevideos', host='localhost', database='crop_videos')
cursor = cnx.cursor()

import re

def format_date_str(date_str):
    """Convert date string in format "Mon_DDth__YYYY" to "YYYY-MM-DD"."""
    match = re.match(r"(\w{3})_(\d{1,2})(?:st|nd|rd|th)__(\d{4})", date_str)
    month_str_to_num = {
        'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04', 'May': '05', 'Jun': '06',
        'Jul': '07', 'Aug': '08', 'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'
    }

    if match:
        month = month_str_to_num[match.group(1)]
        day = match.group(2).zfill(2)  # Making sure it's two digits
        year = match.group(3)
        return f"{year}-{month}-{day}"
    else:
        raise ValueError("Invalid date string format")

# Example Usage
formatted_date = format_date_str(date_str)
query = f"SELECT * FROM video WHERE create_date='{formatted_date}' ORDER BY create_time ASC"
cursor.execute(query)

videos_from_db = cursor.fetchall()

def extract_speed(s):
    try:
        # Split the string by space and take the first part
        return float(s.split(' ')[0])
    except:
        return None

if len(videos_from_db) != len(all_wind_data):
    raise ValueError("The lengths of videos_from_db and all_wind_data are not the same!")

# 4. Map wind data with videos
mapped_data = []
for video, wind_data in zip(videos_from_db, all_wind_data):
    # Apply the function to the 'Wind Speed' column
    wind_data['Wind Speed'] = wind_data['Wind Speed'].apply(extract_speed)
    mapped_data.append((video, wind_data))
    
def average_wind_direction(wind_data: pd.DataFrame, column_name: str = 'Wind Direction') -> float:
    """
    Calculate the average wind direction.
    
    Parameters:
    - wind_data: DataFrame containing wind data.
    - column_name: The column in the DataFrame that contains the wind direction values in degrees.
    
    Returns:
    - The average wind direction in degrees.
    """
    
    # Convert the wind direction to radians
    wind_data['Wind Direction Radians'] = np.deg2rad(wind_data[column_name])

    # Convert wind direction to Cartesian coordinates
    wind_data['x'] = np.cos(wind_data['Wind Direction Radians'])
    wind_data['y'] = np.sin(wind_data['Wind Direction Radians'])

    # Compute the mean x and y components
    mean_x = wind_data['x'].mean()
    mean_y = wind_data['y'].mean()

    # Convert the mean x and y components back to a polar coordinate for the average direction
    avg_direction = np.rad2deg(np.arctan2(mean_y, mean_x))
    
    # Ensure the direction is between 0 and 360 degrees
    if avg_direction < 0:
        avg_direction += 360
        
    # Drop temporary columns
    wind_data.drop(['Wind Direction Radians', 'x', 'y'], axis=1, inplace=True)
    
    return avg_direction

# Insert wind data into the database and update the video entry
for video, wind_data in mapped_data:
    video_info = f"Video created on {video[1]} at {video[2]} with duration {video[5]}"
    print(video_info)
    
    # If wind_data DataFrame is empty, set averages to NULL
    if wind_data.empty:
        avg_speed = None
        avg_direction = None
    else:
        avg_speed = wind_data['Wind Speed'].mean()
        avg_direction = average_wind_direction(wind_data)

    # Convert wind_data DataFrame to a serialized format (e.g., JSON)
    serialized_wind_data = wind_data.to_json()

    # Insert into wind_data table
    insert_query = """
    INSERT INTO wind_data (wind_speed_avg, wind_direction_avg, detailed_wind_change_table, video_id)
    VALUES (%s, %s, %s, %s)
    """
    cursor.execute(insert_query, (avg_speed, avg_direction, serialized_wind_data, video[0]))  # Assuming video[0] is video_id
    cnx.commit()

    # Get the last inserted id
    wind_data_id = cursor.lastrowid

    # Update the video entry with wind_data_id
    update_query = "UPDATE video SET wind_data_id = %s WHERE video_id = %s"
    cursor.execute(update_query, (wind_data_id, video[0]))
    cnx.commit()

    print(wind_data[['Flight time', 'Altitude', 'Wind Direction', 'Wind Speed']])
    print("-" * 40)

# Close the cursor and connection
cursor.close()
cnx.close()


C:\Users\46596\AppData\Local\Temp\ipykernel_6760\2207314937.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wind_data['Wind Speed'] = wind_data['Wind Speed'].apply(extract_speed)


Video created on DJI_0004.MP4 at \\sf3.bss.phy.private.cam.ac.uk\cicutagroup\crop_data\wheat_videos\drone_videos\original\DJI_0004.MP4 with duration 55.0


OperationalError: 2013 (HY000): Lost connection to MySQL server during query

In [8]:
# Imports and defines for loading the wind data
###############################################
import os
import pandas as pd
import mysql.connector
import numpy as np
import re

def extract_time_minutes_seconds(timestr):
    mins, secs = timestr.split('m')
    return int(mins.strip()), int(secs.split('s')[0].strip())

def format_date_str(date_str):
    match = re.match(r"(\w{3})_(\d{1,2})(?:st|nd|rd|th)__(\d{4})", date_str)
    month_str_to_num = {
        'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04', 'May': '05', 'Jun': '06',
        'Jul': '07', 'Aug': '08', 'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'
    }
    if match:
        month = month_str_to_num[match.group(1)]
        day = match.group(2).zfill(2)
        year = match.group(3)
        return f"{year}-{month}-{day}"
    else:
        raise ValueError("Invalid date string format")

def extract_speed(s):
    try:
        return float(s.split(' ')[0])
    except:
        return None

def average_wind_direction(wind_data: pd.DataFrame, column_name: str = 'Wind Direction') -> float:
    wind_data['Wind Direction Radians'] = np.deg2rad(wind_data[column_name])
    wind_data['x'] = np.cos(wind_data['Wind Direction Radians'])
    wind_data['y'] = np.sin(wind_data['Wind Direction Radians'])
    mean_x = wind_data['x'].mean()
    mean_y = wind_data['y'].mean()
    avg_direction = np.rad2deg(np.arctan2(mean_y, mean_x))
    if avg_direction < 0:
        avg_direction += 360
    wind_data.drop(['Wind Direction Radians', 'x', 'y'], axis=1, inplace=True)
    return avg_direction

def extract_date_from_filename(filename):
    """Extract date string from filename"""
    match = re.match(r"(\w{3}_\d{1,2}(?:st|nd|rd|th)__\d{4})", filename)
    if match:
        return match.group(1)
    else:
        return None

In [11]:
# For all dates, match the wind data with each video according to the media data
# Update the wind data into the database and link them to the videos
####################################################################
from colorama import init, Fore
# Initialize colorama
init(autoreset=True)

# Connect to database
cnx = mysql.connector.connect(user='root', password='dronevideos', host='localhost', database='crop_videos')
cursor = cnx.cursor()

media_folder = '\\sf3.bss.phy.private.cam.ac.uk\\cicutagroup\\crop_data\\airdata\\media_data'
wind_folder = '\\sf3.bss.phy.private.cam.ac.uk\\cicutagroup\\crop_data\\airdata\\wind_data'
# Extract unique dates from media files
all_dates = sorted(list(set([extract_date_from_filename(f) for f in os.listdir(media_folder) if extract_date_from_filename(f) is not None])))

for date_str in all_dates:
    print(f"Processing data for {date_str}...")
    media_files = sorted([f for f in os.listdir(media_folder) if date_str in f])
    wind_files = sorted([f for f in os.listdir(wind_folder) if date_str in f])

    # # Gather files for a specific date

    # date_str = "Jul_9th__2022"
    # media_files = sorted([f for f in os.listdir(media_folder) if date_str in f])
    # wind_files = sorted([f for f in os.listdir(wind_folder) if date_str in f])

    # Extract wind data for each video
    all_wind_data = []
    for media_file in media_files:
        df_media = pd.read_csv(os.path.join(media_folder, media_file))
        video_starts = df_media[df_media['Type'] == '    Video Started    ']['Flight time'].tolist()
        video_ends = df_media[df_media['Type'] == '    Video Ended    ']['Flight time'].tolist()
        df_wind = pd.read_csv(os.path.join(wind_folder, media_file))
        video_wind_data_list = []
        for start, end in zip(video_starts, video_ends):
            start_min, start_sec = extract_time_minutes_seconds(start)
            end_min, end_sec = extract_time_minutes_seconds(end)
            df_wind['TimeInMinutes'] = df_wind['Flight time'].apply(lambda x: extract_time_minutes_seconds(x)[0])
            df_wind['TimeInSeconds'] = df_wind['Flight time'].apply(lambda x: extract_time_minutes_seconds(x)[1])
            filtered_wind_data = df_wind[
                (df_wind['TimeInMinutes'] * 60 + df_wind['TimeInSeconds'] >= start_min * 60 + start_sec - 1) &
                (df_wind['TimeInMinutes'] * 60 + df_wind['TimeInSeconds'] <= end_min * 60 + end_sec + 1)
            ]
            video_wind_data_list.append(filtered_wind_data)
        all_wind_data.extend(video_wind_data_list)

    # Fetch all videos from the database
    formatted_date = format_date_str(date_str)
    query = f"SELECT * FROM video WHERE create_date='{formatted_date}' ORDER BY create_time ASC"
    cursor.execute(query)
    videos_from_db = cursor.fetchall()

    if len(videos_from_db) != len(all_wind_data):
        print(Fore.RED + "The lengths of videos_from_db and all_wind_data are not the same!")
        # No need to raise an error, just print it and continue with the next steps

    # Map wind data with videos
    mapped_data = []
    for video, wind_data in zip(videos_from_db, all_wind_data):
        wind_data['Wind Speed'] = wind_data['Wind Speed'].apply(extract_speed)
        mapped_data.append((video, wind_data))

    # Insert wind data into the database and update the video entry
    for video, wind_data in mapped_data:
        # Check if wind data for this video already exists
        check_query = "SELECT COUNT(*) FROM wind_data WHERE video_id = %s"
        cursor.execute(check_query, (video[0],))
        count = cursor.fetchone()[0]
        if count > 0:
            print(f"Wind data for video_id {video[0]} already exists. Skipping...")
            continue
        
        video_info = f"Video created on {video[1]} at {video[2]} with duration {video[5]}"
        print(video_info)
        if wind_data.empty:
            avg_speed = None
            avg_direction = None
        else:
            avg_speed = round(wind_data['Wind Speed'].mean(),2)
            avg_direction = round(average_wind_direction(wind_data),2)
        serialized_wind_data = wind_data.to_json()
        insert_query = """
        INSERT INTO wind_data (wind_speed_avg, wind_direction_avg, detailed_wind_change_table, video_id)
        VALUES (%s, %s, %s, %s)
        """
        cursor.execute(insert_query, (avg_speed, avg_direction, serialized_wind_data, video[0]))
        cnx.commit()
        wind_data_id = cursor.lastrowid
        update_query = "UPDATE video SET wind_data_id = %s WHERE video_id = %s"
        cursor.execute(update_query, (wind_data_id, video[0]))
        cnx.commit()
        print(wind_data[['Flight time', 'Altitude', 'Wind Direction', 'Wind Speed']])
        print("-" * 40)

# Close the cursor and connection
cursor.close()
cnx.close()


Processing data for Jul_10th__2023...


C:\Users\46596\AppData\Local\Temp\ipykernel_6760\1153199859.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wind_data['Wind Speed'] = wind_data['Wind Speed'].apply(extract_speed)


Wind data for video_id 745 already exists. Skipping...
Wind data for video_id 604 already exists. Skipping...
Wind data for video_id 529 already exists. Skipping...
Wind data for video_id 542 already exists. Skipping...
Wind data for video_id 526 already exists. Skipping...
Wind data for video_id 688 already exists. Skipping...
Wind data for video_id 706 already exists. Skipping...
Wind data for video_id 459 already exists. Skipping...
Wind data for video_id 692 already exists. Skipping...
Wind data for video_id 457 already exists. Skipping...
Wind data for video_id 544 already exists. Skipping...
Wind data for video_id 546 already exists. Skipping...
Wind data for video_id 699 already exists. Skipping...
Wind data for video_id 508 already exists. Skipping...
Wind data for video_id 607 already exists. Skipping...
Wind data for video_id 658 already exists. Skipping...
Wind data for video_id 582 already exists. Skipping...
Wind data for video_id 503 already exists. Skipping...
Wind data 

C:\Users\46596\AppData\Local\Temp\ipykernel_6760\1153199859.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wind_data['Wind Speed'] = wind_data['Wind Speed'].apply(extract_speed)


Wind data for video_id 718 already exists. Skipping...
Wind data for video_id 771 already exists. Skipping...
Wind data for video_id 629 already exists. Skipping...
Wind data for video_id 560 already exists. Skipping...
Wind data for video_id 701 already exists. Skipping...
Wind data for video_id 561 already exists. Skipping...
Wind data for video_id 700 already exists. Skipping...
Wind data for video_id 641 already exists. Skipping...
Wind data for video_id 655 already exists. Skipping...
Wind data for video_id 458 already exists. Skipping...
Wind data for video_id 592 already exists. Skipping...
Wind data for video_id 705 already exists. Skipping...
Wind data for video_id 485 already exists. Skipping...
Wind data for video_id 649 already exists. Skipping...
Wind data for video_id 709 already exists. Skipping...
Wind data for video_id 533 already exists. Skipping...
Wind data for video_id 653 already exists. Skipping...
Wind data for video_id 473 already exists. Skipping...
Wind data 

...

   Flight time Altitude  Wind Direction  Wind Speed
85     10m 15s    9.5 m              46        5.75
86     10m 20s    9.4 m              49        5.47
----------------------------------------
Video created on DJI_0222.MP4 at \\sf3.bss.phy.private.cam.ac.uk\cicutagroup\crop_data\wheat_videos\drone_video_525_623\original\DJI_0222.MP4 with duration 11.91


C:\Users\46596\AppData\Local\Temp\ipykernel_6760\248383614.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wind_data['Wind Direction Radians'] = np.deg2rad(wind_data[column_name])
C:\Users\46596\AppData\Local\Temp\ipykernel_6760\248383614.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wind_data['x'] = np.cos(wind_data['Wind Direction Radians'])
C:\Users\46596\AppData\Local\Temp\ipykernel_6760\248383614.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

   Flight time Altitude  Wind Direction  Wind Speed
87     10m 25s   10.0 m              40        5.36
88     10m 30s   10.2 m              52        7.26
89     10m 35s   10.3 m              56        6.83
----------------------------------------
Video created on DJI_0223.MP4 at \\sf3.bss.phy.private.cam.ac.uk\cicutagroup\crop_data\wheat_videos\drone_video_525_623\original\DJI_0223.MP4 with duration 10.44


C:\Users\46596\AppData\Local\Temp\ipykernel_6760\248383614.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wind_data['Wind Direction Radians'] = np.deg2rad(wind_data[column_name])
C:\Users\46596\AppData\Local\Temp\ipykernel_6760\248383614.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wind_data['x'] = np.cos(wind_data['Wind Direction Radians'])
C:\Users\46596\AppData\Local\Temp\ipykernel_6760\248383614.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

   Flight time Altitude  Wind Direction  Wind Speed
89     10m 35s   10.3 m              56        6.83
90     10m 40s   10.3 m              56        5.67
----------------------------------------


In [2]:
# Add video_id as a foreign key in wind_data table
##################################################
import mysql.connector

# Connect to the database
cnx = mysql.connector.connect(user='root', password='dronevideos', host='localhost', database='crop_videos')
cursor = cnx.cursor()

# Step 1: Check if the video_id column exists in the wind_data table
cursor.execute("""
SHOW COLUMNS FROM wind_data LIKE 'video_id';
""")
result = cursor.fetchone()

# Step 2: If the video_id column does not exist, then add it
if not result:
    cursor.execute("""
    ALTER TABLE wind_data
    ADD COLUMN video_id INT;
    """)

# Step 3: Define video_id in wind_data as a foreign key referencing video_id in video table
cursor.execute("""
ALTER TABLE wind_data
ADD CONSTRAINT fk_video_id
FOREIGN KEY (video_id) REFERENCES video(video_id);
""")

# Commit the transaction
cnx.commit()

# Close the cursor and connection
cursor.close()
cnx.close()

In [3]:
# Print all foreign keys in video table
#######################################
import mysql.connector

# Connect to the database
cnx = mysql.connector.connect(user='root', password='dronevideos', host='localhost', database='crop_videos')
cursor = cnx.cursor()

# Query to fetch all foreign keys in the video table
query = """
SELECT COLUMN_NAME
FROM INFORMATION_SCHEMA.KEY_COLUMN_USAGE
WHERE TABLE_NAME = 'video'
AND TABLE_SCHEMA = 'crop_videos'
AND REFERENCED_TABLE_NAME IS NOT NULL;
"""

cursor.execute(query)
foreign_keys = cursor.fetchall()

# Check and print the results
if foreign_keys:
    print("Foreign Key columns in 'video' table:")
    for fk in foreign_keys:
        print(f"- {fk[0]}")
else:
    print("No foreign key columns found in 'video' table.")

# Close the cursor and connection
cursor.close()
cnx.close()


Foreign Key columns in 'video' table:
- drone_id
- wind_data_id
- wheat_data_id
- video_group_id
